In [ ]:
import pandas as pd
from openprompt.data_utils import InputExample

# Load data
df = pd.read_csv(
    r"C:\Users\stdFurqan\Desktop\paft\XNLI\XNLI_test_urdu - Sheet1 - XNLI_test_urdu - Sheet1.tsv.csv"
)

# Fixed label mapping
label_map = {
    "entailment": 0,
    "neutral": 1,
    "contradiction": 2
}
classes = list(label_map.keys())

guid = 0
examples = []
class_stats = {}

# Take FIRST 16 examples per class
for label_name in classes:
    class_df = df[df["gold_label"] == label_name].head(16)
    class_stats[label_name] = len(class_df)

    for _, row in class_df.iterrows():
        examples.append(
            InputExample(
                guid=guid,
                text_a=str(row["sentence1"]).strip(),
                text_b=str(row["sentence2"]).strip(),
                label=label_map[label_name]
            )
        )
        guid += 1

# -------------------------------
# Print exactly like you want
# -------------------------------
print("### InputExamples ###\n")
for ex in examples:
    print(
        "InputExample(\n"
        f"    guid={ex.guid},\n"
        f"    text_a=\"{ex.text_a}\",\n"
        f"    text_b=\"{ex.text_b}\",\n"
        f"    label={ex.label}\n"
        "),"
    )

# -------------------------------
# Metadata
# -------------------------------
print("\n### Classes ###")
print(f"classes = {classes}")

print("\n### Label Map ###")
print(f"label_map = {label_map}")

print("\n### Samples per Class ###")
for cls, count in class_stats.items():
    print(f"{cls}: {count}")

print(f"\nTotal InputExamples generated: {guid}")


In [ ]:
templates = [
    ("P1", ManualTemplate(
        text='{"placeholder":"text_a"} اور {"placeholder":"text_b"} کا تعلق {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),

    ("P2", ManualTemplate(
        text='پہلا جملہ: {"placeholder":"text_a"} دوسرا جملہ: {"placeholder":"text_b"} ان دونوں کے درمیان تعلق {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),

    ("P3", ManualTemplate(
        text='{"placeholder":"text_a"} کے مقابلے میں {"placeholder":"text_b"} کا مفہوم {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),

    ("P4", ManualTemplate(
        text='اگر {"placeholder":"text_a"} درست ہے تو {"placeholder":"text_b"} {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),

    ("P5", ManualTemplate(
        text='{"placeholder":"text_b"} کا بیان {"placeholder":"text_a"} کے لحاظ سے {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),

    ("P6", ManualTemplate(
        text='{"placeholder":"text_a"} اور {"placeholder":"text_b"} کے درمیان رشتہ {"mask"} قرار پاتا ہے۔',
        tokenizer=tokenizer,
    )),

    ("P7", ManualTemplate(
        text='پہلے جملے {"placeholder":"text_a"} کی بنیاد پر دوسرا جملہ {"placeholder":"text_b"} {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),

    ("P8", ManualTemplate(
        text='{"placeholder":"text_a"} کے تناظر میں {"placeholder":"text_b"} کا تعلق {"mask"} بنتا ہے۔',
        tokenizer=tokenizer,
    )),

    ("P9", ManualTemplate(
        text='{"placeholder":"text_a"} کے بعد {"placeholder":"text_b"} کا نتیجہ {"mask"} سمجھا جاتا ہے۔',
        tokenizer=tokenizer,
    )),

    ("P10", ManualTemplate(
        text='{"placeholder":"text_a"} اور {"placeholder":"text_b"} کے درمیان منطقی تعلق {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),
]


In [1]:
# Set device
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
# ==============================
# Imports
# ==============================
import torch
import pandas as pd
import random
import numpy as np
from collections import Counter
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
from openprompt.prompts import ManualTemplate, ManualVerbalizer
from openprompt.data_utils import InputExample
from openprompt.plms import load_plm
from openprompt import PromptForClassification, PromptDataLoader
from torch.optim import AdamW
from sklearn.metrics import classification_report
from collections import defaultdict
from torch.utils.data import DataLoader, Sampler

# ========================================
# Check CUDA
# ========================================
device = "cuda" #if torch.cuda.is_available() else "cpu"
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))
    print("CUDA version:", torch.version.cuda)
    print("GPU count:", torch.cuda.device_count())

# ========================================
# Seeds for reproducibility
# ========================================
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA available: True
GPU name: NVIDIA GeForce RTX 4080 SUPER
CUDA version: 12.1
GPU count: 1


In [3]:

# Step 1: Training data (16-shot) ha,m and spam
train_dataset = [

            InputExample(
            guid=0,
            text_a="یہ تہوار تین سے چار دن سے منایا جاتا ہے، جس میں کئی کروڑ قریبی قسط واپس آنے کے لۓ.",
            text_b="- تہوار ایک ہفتےکے اندر ہوگا",
            label=0
        ),
        InputExample(
            guid=1,
            text_a="غیر ملکیوں نے ملک چھوڑنے کے بعد نمائندوں کو ختم کرنے کے بارے میں کئی گواہوں کی گواہی دی.",
            text_b=",ډیری شاهدانو نماینده ګانې رد کړه",
            label=0
        ),
        InputExample(
            guid=2,
            text_a="..آپ کا حصہ بننے پر آپ پر فخر ہو گی؟",
            text_b="?ایک آپ بھی تعاون رکن ہو گے",
            label=0
        ),
        InputExample(
            guid=3,
            text_a="یہ ملک سب کچھ ہے سر۔ حاکمیت کچھ نہیں ۔",
            text_b=". یہ ملک خود مختار نہیں ہے.",
            label=0
        ),
        InputExample(
            guid=4,
            text_a="یہ افسر، جس نے جنوبی ٹاور کے خاتمے کا مشاہدہ کیا تھا، اس کے انکشافی ہدایات میں شمال ٹاور میں ESU یونٹس کو بتایا.",
            text_b=".افسر د سویلي ټاور د ړنګ ګواه وو",
            label=0
        ),
        InputExample(
            guid=5,
            text_a="آپ جانتے ہیں کہ وہ چلیں جایں گے اور اتنی سرگرمیاں نہیں ہوں گی، کیا آپ",
            text_b=".ایک دفعہ وہ چھوڑ دیں گے، سرگرمیاں بھی ہو گی",
            label=0
        ),
        InputExample(
            guid=6,
            text_a="نوبت یہاں تک پہنچ گئی ہے کہ ایک ہفتے میں دو یا تین ہوائی جہاز آتے ہیں اور مجھے نہیں پتہ کہ وہ اڑ کر کہاں جا رہے ہیں.",
            text_b=".ایک سے زیادہ طیارے ہر ہفتے پہنچتے ہیں",
            label=0
        ),
        InputExample(
            guid=7,
            text_a="طویل عرصے سے، معلومات کے دفتر کو انٹرکولنگ واٹر ویز کے کوئٹیکو کی شاندار نیٹ ورک کے تفصیلی نقشے فراہم کرتی ہیں",
            text_b=".په کویټکو کي ډیري د اوبو لاري دي",
            label=0
        ),
        InputExample(
            guid=8,
            text_a="مثال کے طور پر، زیادہ تر، تمام جینز جامنی رنگ میں تبدیل ہو جاتے ہیں.",
            text_b=".جینز کا رنگ تبدیل کرنا ممکن ہے",
            label=0
        ),
        InputExample(
            guid=9,
            text_a="کچھ امریکی جگہ کے نام ان کے بارے میں ایک منفرد گونج رکھتے ہیں - میگگی کے نپلس، وومومنگ، یا گرسی کریک، آرکنساس، لنکسلیٹ، کینٹکی، یا سکرو لانٹ آؤٹ، الاباما جیسے مقامات",
            text_b=".ځینې نومونه یو ځانګړي تعلق لري",
            label=0
        ),
        InputExample(
            guid=10,
            text_a="معلومات کے لیے‏، اس نمبر (213) 623-2489 پر کال کریں ہفتہ کے علاوہ کسی دن 9 بجے صبح سے 5 بجے شام کے درمیان۔",
            text_b=".د تليفون کرښه په اونۍ کې بوخوته وي",
            label=0
        ),
        InputExample(
            guid=11,
            text_a="، چھوٹے طبقات، ٹیکنالوجی کا استعمال)، اور طلباء کی طویل مدت سے حمایت کی کمی (لاکرس، فوڈ سروس، طالب علم تنظیموں کے دفاتر).",
            text_b=".د زده کونکو لپاره د لاکرو نشتوالي شته",
            label=0
        ),
        InputExample(
            guid=12,
            text_a="کہانی کا اظہار کرنے کے لئے یہ ایک شاندار وقت ہے.",
            text_b=".دا یو ښه وخت دی چې د کیسې وکړو",
            label=0
        ),
        InputExample(
            guid=13,
            text_a="اگر اور اس منصوبے کو مکمل کیا جائے تو، یہ پوری سلسلہ کا سب سے زیادہ دلچسپ ہونا چاہئے.",
            text_b=".دا یوه زړه پورې پروژه ده",
            label=0
        ),
        InputExample(
            guid=14,
            text_a="جھیل کے جنوبی ساحل پہ سب سے بڑا شہر سیفوک ہے.",
            text_b=".سوفک جنوبی ساحل پر ہے.",
            label=0
        ),
        InputExample(
            guid=15,
            text_a="نومبرمیں ھم نے خط بھیجے تھے جس میں آپ کے ساتھ لڑکوں اور لڑکیوں کی خیمہ بستی جو کے بچوں اور نوجوانوں کے لئے ایک بہت عمدہ اور مثبت جگہ ھے، کے بارے میں کہانی بتائی تھی۔",
            text_b=".موږ په نومبر کې لیکونه باهر ته لیږو.",
            label=0
        ),
        InputExample(
            guid=16,
            text_a="تمام ضروری حصوں یا عناصر کو شامل کرنا.",
            text_b="-انہیں نئے راکٹ کے حصوں میں شامل کرنا ہوگا",
            label=1
        ),
        InputExample(
            guid=17,
            text_a="ہمیں یہی مشورہ دیا گیا تھا.",
            text_b=",کله چې ویل کیده چې څه وکړي اداره ناکامه شوه چي موږ دننه د .سوداګري رازونو ته راپریږدي",
            label=1
        ),
        InputExample(
            guid=18,
            text_a="تو سلیٹ کے پاس کتنے قارئین ہیں؟",
            text_b="،سټیټ یو بلیون لوستونکي لريسمه ده؟",
            label=1
        ),
        InputExample(
            guid=19,
            text_a="روحانی رہنمائی اور حوصلہ افزائی.",
            text_b="..محبت کے بارے میں مشورہ",
            label=1
        ),
        InputExample(
            guid=20,
            text_a="ڈو 3 ملیگرام ایک رات بہت زیادہ ہے.",
            text_b=".د شپې 1 ملی ګرامه یو ښه خوراک دي",
            label=1
        ),
        InputExample(
            guid=21,
            text_a="اہم Italian کتاب Rome کی رہنمائی دعویٰ کرتی ہے کہ یہ عمارت کا دوسرا نام II Colosseo Quadrato، Chaqoor Colosseum رکھا گیا ہے۔",
            text_b=".د ودانی تعاقب کیږی",
            label=1
        ),
        InputExample(
            guid=22,
            text_a="مساوات کے بارے میں امریکی نقطہ نظر کا اختلاف یہ ہے کہ اگرچہ ہم معاشی مساوات اور امتیازی امتیازی سلوک کے بارے میں ہماری تشویش میں یورپی معاشرے کو پیار کرتے ہیں، ہم دنیا کے مساوات کے دوسرے علاقوں میں دنیا کی قیادت کرتے ہیں",
            text_b=".یورپی معاشرے اقتصادی مساوات کے بارے میں تشویش میں گلوبل رہنما ہیں.",
            label=1
        ),
        InputExample(
            guid=23,
            text_a="200,000 الفاظ کی novel $25 کے حساب سے 8,000 الفاظ پر dollar ہو گئے۔",
            text_b="$ 25 میں ایک 200،000 لفظ ناول منصفانہ قیمت ہے",
            label=1
        ),
        InputExample(
            guid=24,
            text_a="یہ اہم عوامی ساحلوں (وارووک لانگ خ سے ہورسیس بے) سے منسلک ہیں.",
            text_b="$ 5 کے لئے عوامی ساحل سمندر.",
            label=1
        ),
        InputExample(
            guid=25,
            text_a="شکایات، سامان کے مسائل، بد مزاج فضائی میزبان، پراسرار طور پر منسوخ شدہ پروازوں اور ادائیگی میں کسی قسم کی زیادتی پر مشتمل ہیں۔",
            text_b="10 لوگوں کا سامان کھو گیا تھا۔",
            label=1
        ),
        InputExample(
            guid=26,
            text_a="لیکن بہت سے ایسے ہیں جو اب بھی ہماری مدد کی ضرورت ہے.",
            text_b="10،000 افراد کو اب بھی ہماری مدد کی ضرورت ہے.",
            label=1
        ),
        InputExample(
            guid=27,
            text_a="80 فیصد شرکاء تنازعات کے حل کی مہارت میں اضافہ پر رپورٹ کریں گے.",
            text_b="100 انفرادی شرکاء تھے",
            label=1
        ),
        InputExample(
            guid=28,
            text_a="اگر ہم سوچتے ہیں کہ ہم 1787، 1791 ء یا 1868 میں غالب ہوئے دنیا پر ایک خاص قدم اٹھاتے ہیں، تو ہمیں یہ فیصلہ کرنا چاہئے کہ جن کی جذبات کا معاملہ ہے.",
            text_b="1787 میں لوگ سوچتے ہیں کہ دنیا فلیٹ تھی",
            label=1
        ),
        InputExample(
            guid=29,
            text_a="1863 میں قوم میں ابھی بھی ایک زیادہ کامل اتحادی بنانے کی خواہش ہے، لیکن اس کے علاوہ اس میں ماضی بھی تھا جس نے دونوں کی حوصلہ افزائی کی اور نئی دیسی سائیکی کو ریشانی میں مبتلا کر دیا۔",
            text_b="1863 ء میں سیاسی بدامنی کی وجہ سے ملک بدل رہا تھا.",
            label=1
        ),
        InputExample(
            guid=30,
            text_a="بٹھانیہ 1896 نیو گوتھائی کیلونسٹ چرچ کے رنگا رنگ کے بیرونی حصے کی تعریف کرنے سے روکنے سے قبل روانہ ہو جاؤ، جو شہر کے بہت سے پینوراموں میں ہے.",
            text_b="1896 نوگو گوتھائی کیلنسٹسٹ چرچ چند پینوراما ہے.",
            label=1
        ),
        InputExample(
            guid=31,
            text_a="اگر ایک 1984 کا ایڈیشن تھا تو، شاید اس سے بکرفر (اور کم مہنگی) تکمیل کے مقابلے میں اس کتاب کو خریدنے کے لئے بہت خوش رہ سکتا ہے.",
            text_b="1984 کا ایڈیشن بہت اچھا ہے.",
            label=1
        ),
        InputExample(
            guid=32,
            text_a="پچھلے صدیوں میں، تاہم، یہ کیریبین قزاقوں کا شکار تھا،ہننا میں نوآبادیاتی اداروں کے چشموں سے دور،پیٹوٹو ریکو اور پاناما سٹی میں قریبی نوآبادیاتی جگہوں پر سان جوآن.",
            text_b="-کیریبین میں کبھی قزاقوں نہیں تھے",
            label=2
        ),
        InputExample(
            guid=33,
            text_a="یہ آج ہم زندہ راستہ ہے، گیری کہہ رہا ہے، کیوں اس سے لطف اندوز نہیں ہے؟",
            text_b="-گہری کو خوشی کی فکر نہیں",
            label=2
        ),
        InputExample(
            guid=34,
            text_a="لیکن اب میکسیل نے ایک مکھی مخلوق میں داخل ہوکر اس کو مدعو کیا، بعد میں میکسیل کے راکشس کو ڈوب دیا.",
            text_b=",د مهارت نشتوالی له کبله میکس ویل هیڅکله په خپل ټول ژوند کي .هیڅح شی نه دی ایجاد کړي",
            label=2
        ),
        InputExample(
            guid=35,
            text_a="اگر اور اس منصوبے کو مکمل کیا جائے تو، یہ پوری سلسلہ کا سب سے زیادہ دلچسپ ہونا چاہئے.",
            text_b="!یہ بہت بورنگ ہے",
            label=2
        ),
        InputExample(
            guid=36,
            text_a="روحانی رہنمائی اور حوصلہ افزائی.",
            text_b="..تھیوں سے رہنمائی",
            label=2
        ),
        InputExample(
            guid=37,
            text_a="ونڈوز کے چلتے ہوئے آپ ان ڈاٹ ڈی ایل ایل (dll.) فائلوں کو حدف نہیں کر سکتے (جو کے مائیکروسافٹ کے نقطے کا حصہ ہے).",
            text_b="فائلوں کا آپریٹنگ سسٹم پر کوئی اثر نہیں ہوتا اور آپ جب چاہیں ان کو ڈلیٹ کر سکتے ہیں۔",
            label=2
        ),
        InputExample(
            guid=38,
            text_a="یہ کال ٹرمینل سی میں ایک پی سی او فون سے کی گئی تھی (اسکریننگ چیک پوائنٹ اوریونائیٹڈ 175 کے بورڈنگ دروازے کے درمیان )",
            text_b=".ټرمینل سی په بشپړ ډول خالي شو",
            label=2
        ),
        InputExample(
            guid=39,
            text_a="آپ‏ دیگر مشفق ممبران کے ساتھ ہمارے اسٹیٹ کے باوقار اور عالی ورثہ کو فروغ دینے اور نقصان یا تباہی سے بچانے کے لیے کوشش میں مدد کررہے ہوں گے۔",
            text_b=".دولت د چا د مرستي پرته ښه کار کوي",
            label=2
        ),
        InputExample(
            guid=40,
            text_a="میں اس طرح کے دورانیہ کے واپس مسائل کے ذریعے پڑھنے سے بہت لطف اندوز ہوسکتا ہوں، جب آپ سمجھتے ہیں کہ ہر سال تقریبا چار سے زائد صفحات کے دو حجم بن جاتے ہیں.",
            text_b=".زه یواځې د اوسني مسلو لوستل کوم",
            label=2
        ),
        InputExample(
            guid=41,
            text_a="مالک، وہ تقسیم شخصیت شخصیت آدمی کی طرح تھا.",
            text_b=".میرا مالک پیار اور مستقل تھا",
            label=2
        ),
        InputExample(
            guid=42,
            text_a="گلیسی آبجیکٹ کی شدت پسندوں نے انہیں بالکل فکر نہیں کی، اور اس کی بہترین وجہ سے وہ کوشش کرنے پر مجبور نہیں کرتا.",
            text_b=".هغه په نړۍ کې ټولې اندیښنې لري",
            label=2
        ),
        InputExample(
            guid=43,
            text_a="خیر، ہماری تعیناتی ہوئی اور میں اب بتا سکتا ہوں کہ اس کی وجہ سے ہماری کادینا، اوکیناوا میں تعیناتی ہوئی 1968 میں.",
            text_b=".ہم نے کبھی کسی کو تعینات نہیں کیا",
            label=2
        ),
        InputExample(
            guid=44,
            text_a="لہذا یہ افراتفری کے علاوہ کچھ نہیں دیتا۔",
            text_b=".یہ بہت پرسکون ہے اور کوئی مسئلہ نہیں کرتا ہے",
            label=2
        ),
        InputExample(
            guid=45,
            text_a="اسٹریٹ آف دا سنو' ایک اور قدیم مجسمے کے نام پر رکھا گیا ہے جوکہ ابھی بھی وہاں محفوظ ہے.",
            text_b="۔ڈیلا سکروفا ایک شہر کے بعد نامزد ہے",
            label=2
        ),
        InputExample(
            guid=46,
            text_a="ہاتھ سے بنایا کاریں شہر کا سب سے بہترین پیارا تعلق ہے،اور نظام کو 1 964 میں ایک قومی تاریخی تاریخی نشان قرار دیا گیا تھا.",
            text_b="۔کوئی بھی گاڑیوں کے بارے میں پرواہ نہیں کرتے",
            label=2
        ),
        InputExample(
            guid=47,
            text_a="یہاں 1775 میں تھا کہ فورٹ سٹی کیتھرین میں اسٹوروں سے 100 بیرل بندوق سے پراسرار طور پر غائب ہوگیا اور امریکی انقلابیوں کے استعمال کے لئے پابند ایک کشتی پر اپنا راستہ پایا.",
            text_b="100 بیرل بندوق سے کبھی نہیں استعمال کیا گیا تھا.",
            label=2
        ),
        
    
]


In [4]:
### Classes ###
classes = ['entailment', 'neutral', 'contradiction']

### Label Map ###
label_map = {'entailment': 0, 'neutral': 1, 'contradiction': 2}



# # Step 1: Use load_plm with 'roberta' to get the correct WrapperClass
_, _, _, WrapperClass = load_plm("roberta", "roberta-base")  # Just to get the wrapper

# # # Step 2: Manually load XLM-RoBERTa model/tokenizer
model_name = "xlm-roberta-base"
tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
plm = XLMRobertaForMaskedLM.from_pretrained(model_name)

# ==============================
# Load Pretrained Language Model (mBERT)
# ==============================
# plm, tokenizer, model_config, WrapperClass = load_plm("bert", "bert-base-multilingual-cased")

C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
# ==============================
# Define Prompt Template (Manual)
# ==============================
template = ManualTemplate(
    text = '{"placeholder":"text_a"} اور {"placeholder":"text_b"} کا تعلق {"mask"} ہے۔',
    tokenizer=tokenizer,
)









verbalizer = ManualVerbalizer(
    classes=classes,
    label_words={
        "entailment": ["درست", "لازم"],
        "neutral": ["ممکن", "غیر واضح"],
        "contradiction": ["غلط", "متضاد"]
    },
    tokenizer=tokenizer,
)


In [6]:
# ==============================
# Create Prompt Model
# ==============================
prompt_model = PromptForClassification(
    template=template,
    plm=plm,
    verbalizer=verbalizer
)

# ==============================
# DataLoader for Training
# ==============================
train_loader = PromptDataLoader(
    dataset=train_dataset,
    tokenizer=tokenizer,
    template=template,
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=128,
    batch_size=4,
    shuffle=True  # Even though shuffled, reproducibility is preserved by seeding
)

# # ==============================
# # Fine-Tuning the Prompt Model
# # ==============================
prompt_model.train()
optimizer = AdamW(prompt_model.parameters(), lr=1e-5)

for epoch in range(20):
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        logits = prompt_model(batch)
        loss = torch.nn.CrossEntropyLoss()(logits, batch['label'])
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} Loss: {total_loss:.4f}")

tokenizing: 48it [00:00, 1814.18it/s]


Epoch 1 Loss: 18.7164
Epoch 2 Loss: 12.9436
Epoch 3 Loss: 10.8458
Epoch 4 Loss: 8.9539
Epoch 5 Loss: 7.0251
Epoch 6 Loss: 4.1258
Epoch 7 Loss: 2.9022
Epoch 8 Loss: 2.3560
Epoch 9 Loss: 1.0380
Epoch 10 Loss: 0.6379
Epoch 11 Loss: 0.3432
Epoch 12 Loss: 0.3998
Epoch 13 Loss: 0.0935
Epoch 14 Loss: 0.1428
Epoch 15 Loss: 0.0465
Epoch 16 Loss: 0.0472
Epoch 17 Loss: 0.0487
Epoch 18 Loss: 0.0294
Epoch 19 Loss: 0.0210
Epoch 20 Loss: 0.0196


In [7]:
# ==============================
# Load Evaluation Dataset
# ==============================
df = pd.read_csv(
    r"C:\Users\stdFurqan\Desktop\paft\XNLI\XNLI_Dev_Urdu - Sheet1 (1) - XNLI_Dev_Urdu - Sheet1 (1).tsv.csv"
)

# Make InputExamples
eval_dataset = [
    InputExample(
        guid=i,
        text_a=row['sentence1'],
        text_b=row['sentence2'],
        label=label_map[row['gold_label']]
    )
    for i, row in df.iterrows()
]

# ==============================
# PromptDataLoader
# ==============================
eval_loader = PromptDataLoader(
    dataset=eval_dataset,
    tokenizer=tokenizer,
    template=template,              # your ManualTemplate for XNLI
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=128,
    batch_size=8,
    shuffle=False
)

# ==============================
# Evaluate Model
# ==============================
prompt_model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in eval_loader:
        logits = prompt_model(batch)
        preds = torch.argmax(logits, dim=-1)
        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(batch['label'].cpu().tolist())

# ==============================
# Print Classification Report
# ==============================
print("\n📊 XNLI Urdu Dev Classification Report:")
print(classification_report(all_labels, all_preds, target_names=classes, digits=4))

tokenizing: 2490it [00:00, 3314.30it/s]



📊 XNLI Urdu Dev Classification Report:
               precision    recall  f1-score   support

   entailment     0.3787    0.1241    0.1869       830
      neutral     0.3660    0.4675    0.4106       830
contradiction     0.3964    0.5530    0.4618       830

     accuracy                         0.3815      2490
    macro avg     0.3804    0.3815    0.3531      2490
 weighted avg     0.3804    0.3815    0.3531      2490



In [ ]:
📊 XNLI Urdu Dev Classification Report:
               precision    recall  f1-score   support

   entailment     0.3787    0.1241    0.1869       830
      neutral     0.3660    0.4675    0.4106       830
contradiction     0.3964    0.5530    0.4618       830

     accuracy                         0.3815      2490
    macro avg     0.3804    0.3815    0.3531      2490
 weighted avg     0.3804    0.3815    0.3531      2490